In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# The csv file is encoded by ISO-8859-1
data_df = pd.read_csv("Speed Dating Data.csv", encoding="ISO-8859-1")
data_df.head()

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,...,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
0,1,1.0,0,1,1,1,10,7,NaN,4,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
1,1,1.0,0,1,1,1,10,7,NaN,3,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
2,1,1.0,0,1,1,1,10,7,NaN,10,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
3,1,1.0,0,1,1,1,10,7,NaN,5,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
4,1,1.0,0,1,1,1,10,7,NaN,7,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN


In [3]:
data_df.columns.values

array(['iid', 'id', 'gender', 'idg', 'condtn', 'wave', 'round', 'position',
       'positin1', 'order', 'partner', 'pid', 'match', 'int_corr',
       'samerace', 'age_o', 'race_o', 'pf_o_att', 'pf_o_sin', 'pf_o_int',
       'pf_o_fun', 'pf_o_amb', 'pf_o_sha', 'dec_o', 'attr_o', 'sinc_o',
       'intel_o', 'fun_o', 'amb_o', 'shar_o', 'like_o', 'prob_o', 'met_o',
       'age', 'field', 'field_cd', 'undergra', 'mn_sat', 'tuition', 'race',
       'imprace', 'imprelig', 'from', 'zipcode', 'income', 'goal', 'date',
       'go_out', 'career', 'career_c', 'sports', 'tvsports', 'exercise',
       'dining', 'museums', 'art', 'hiking', 'gaming', 'clubbing',
       'reading', 'tv', 'theater', 'movies', 'concerts', 'music',
       'shopping', 'yoga', 'exphappy', 'expnum', 'attr1_1', 'sinc1_1',
       'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1', 'attr4_1', 'sinc4_1',
       'intel4_1', 'fun4_1', 'amb4_1', 'shar4_1', 'attr2_1', 'sinc2_1',
       'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1', 'attr3_1', 'si

In [4]:
data_df[['field','field_cd']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8378 entries, 0 to 8377
Data columns (total 2 columns):
field       8315 non-null object
field_cd    8296 non-null float64
dtypes: float64(1), object(1)
memory usage: 131.0+ KB


In [8]:
f_fcd = data_df[['field','field_cd']].drop_duplicates()
nan_ind = pd.isnull(f_fcd).any(1).nonzero()[0] # Isnull find nan, and any find the row contain nan, nonzero find the index
f_fcd.drop(f_fcd.index[nan_ind], inplace=True)

fcd_ind = pd.isnull(data_df['field_cd']).nonzero()[0]
data_ind = data_df.index
n=0
for i in fcd_ind:
    field_i =data_df.loc[data_ind[i], 'field']
    f_cd = f_fcd[f_fcd['field']==field_i]['field_cd'].values
    if pd.isnull(f_cd) == 0:
        n += 1
        data_df.loc[data_ind[i], 'field_cd'] = f_cd[0]
print("Convert {} field_cd added.".format(n))

Convert 0 field_cd added.


In [66]:
# Check description document to decide the features we can use
# Divide them to group check one by one via count and head
# If the filled data is too few we also don't choose it
use_features = ['gender', 'condtn', 'round', 'order', 'position', 'int_corr', 'samerace', \
                'age_o', 'race_o', 'pf_o_att', 'pf_o_sin', 'pf_o_int','pf_o_fun', 'pf_o_amb', \
                'pf_o_sha', 'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o', 'amb_o', 'shar_o', \
                'like_o', 'prob_o', 'met_o', 'age', 'race', 'imprace', 'imprelig','goal', 'date', \
                'go_out', 'sports', 'tvsports', 'exercise', 'dining', 'museums', 'art', 'hiking', \
                'gaming', 'clubbing', 'reading', 'tv', 'theater', 'movies', 'concerts', 'music', \
                'shopping', 'yoga', 'exphappy', 'attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', \
                'shar1_1', 'attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1', 'attr3_1', \
                'sinc3_1', 'fun3_1', 'intel3_1', 'amb3_1', 'dec', 'attr', 'sinc', 'intel', 'fun', 'amb', \
                'like', 'prob', 'met']
label = ['match']

In [67]:
n_features = len(use_features)
print('Number of features we use: %d' % n_features)

Number of features we use: 76


In [91]:
# Take the data we will use, and check their information
data = data_df[use_features+label]
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8378 entries, 0 to 8377
Data columns (total 77 columns):
gender      8378 non-null int64
condtn      8378 non-null int64
round       8378 non-null int64
order       8378 non-null int64
position    8378 non-null int64
int_corr    8220 non-null float64
samerace    8378 non-null int64
age_o       8274 non-null float64
race_o      8305 non-null float64
pf_o_att    8289 non-null float64
pf_o_sin    8289 non-null float64
pf_o_int    8289 non-null float64
pf_o_fun    8280 non-null float64
pf_o_amb    8271 non-null float64
pf_o_sha    8249 non-null float64
dec_o       8378 non-null int64
attr_o      8166 non-null float64
sinc_o      8091 non-null float64
intel_o     8072 non-null float64
fun_o       8018 non-null float64
amb_o       7656 non-null float64
shar_o      7302 non-null float64
like_o      8128 non-null float64
prob_o      8060 non-null float64
met_o       7993 non-null float64
age         8283 non-null float64
race        8315 non-nu

In [92]:
# All the data are numbers we can use them directly
# However, some of them have few data than others, we need deal nan rows
inds = pd.isnull(data).any(1).nonzero()[0]
print('number of rows contain Nan:', len(inds))

number of rows contain Nan: 2550


In [93]:
# A lot of columns have nan value, here we fill them with average in each columns
data = data.fillna(data.mean())

In [94]:
# Let's check
inds = pd.isnull(data).any(1).nonzero()[0]
print('number of rows contain Nan:', len(inds))

number of rows contain Nan: 0


In [95]:
# Then because we need feed them to NN, it's better to normalize them
from sklearn import preprocessing
minmax_normalizer = preprocessing.MinMaxScaler()
norm_data = pd.DataFrame(minmax_normalizer.fit_transform(data))

In [96]:
norm_data.head()

,0,1,2,3,4,5,6,7,8,9,...,67,68,69,70,71,72,73,74,75,76
0,0.0,0.0,0.294118,0.142857,0.285714,0.557471,0.0,0.243243,0.2,0.35,...,1.0,0.6,0.9,0.7,0.7,0.6,0.7,0.600000,0.250,0.0
1,0.0,0.0,0.294118,0.095238,0.285714,0.787356,0.0,0.108108,0.2,0.60,...,1.0,0.7,0.8,0.7,0.8,0.5,0.7,0.500000,0.125,0.0
2,0.0,0.0,0.294118,0.428571,0.285714,0.568966,1.0,0.108108,0.6,0.19,...,1.0,0.5,0.8,0.9,0.8,0.5,0.7,0.520752,0.125,1.0
3,0.0,0.0,0.294118,0.190476,0.285714,0.827586,0.0,0.135135,0.2,0.30,...,1.0,0.7,0.6,0.8,0.7,0.6,0.7,0.600000,0.250,1.0
4,0.0,0.0,0.294118,0.285714,0.285714,0.597701,0.0,0.162162,0.4,0.30,...,1.0,0.5,0.6,0.7,0.7,0.6,0.6,0.600000,0.250,1.0


## Build Neural network
After we get all the data, we can just build our network.  
We use the neural network from sklean library.
Hyper parameter choose, this is small dataset so we choose solver='lbfgs', activation use default 'relu'.  
And for the hidden layer, because the input layer have 76 units, and output layer have 2 units, so we choose 50 units for hidden layer.  
And we can try more later.

In [130]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

train, test = train_test_split(norm_data, test_size=0.1)
matcher = MLPClassifier(solver='lbfgs', hidden_layer_sizes=(50,), random_state=1)

In [131]:
train.head()

,0,1,2,3,4,5,6,7,8,9,...,67,68,69,70,71,72,73,74,75,76
7922,1.0,1.0,1.000000,0.714286,0.428571,0.517241,0.0,0.135135,0.2,0.2000,...,0.0,0.6,0.6,0.5,0.6,0.4,0.5,0.5,0.00,0.0
3613,0.0,1.0,0.941176,0.238095,0.523810,0.666667,0.0,0.162162,0.4,0.3000,...,0.0,0.4,0.3,0.3,0.3,0.4,0.3,0.3,0.00,0.0
3035,1.0,1.0,0.882353,0.285714,0.380952,0.712644,0.0,0.189189,0.2,0.1538,...,0.0,0.6,0.8,0.9,0.5,0.8,0.6,0.5,0.25,0.0
7199,1.0,1.0,0.588235,0.000000,0.333333,0.356322,0.0,0.324324,0.0,0.1500,...,1.0,0.7,0.6,0.7,0.7,0.7,0.8,0.6,0.00,0.0
3926,0.0,1.0,0.941176,0.190476,0.380952,0.511494,1.0,0.270270,0.4,0.2300,...,1.0,0.7,0.8,0.9,0.6,0.9,0.8,0.5,0.00,1.0


In [132]:
X_train = train.ix[:, :75]
Y_train = train.ix[:, 76]

X_test = test.ix[:, :75]
Y_test = test.ix[:, 76]

In [133]:
Y_train = Y_train.astype(int)
Y_test = Y_test.astype(int)

In [134]:
matcher.fit(X_train, Y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(50,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [135]:
matcher.score(X_test, Y_test)

1.0

In [142]:
predict = matcher.predict(X_test)
print('y_predict\ty')
for i in range(len(Y_test)):
    print(predict[i],'\t',Y_test.values[i])

y_predict	y
1 	 1
0 	 0
0 	 0
1 	 1
1 	 1
1 	 1
0 	 0
0 	 0
0 	 0
0 	 0
1 	 1
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
1 	 1
1 	 1
1 	 1
0 	 0
0 	 0
1 	 1
0 	 0
0 	 0
1 	 1
0 	 0
0 	 0
0 	 0
0 	 0
1 	 1
0 	 0
0 	 0
0 	 0
1 	 1
1 	 1
1 	 1
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
1 	 1
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
1 	 1
1 	 1
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
1 	 1
0 	 0
0 	 0
0 	 0
1 	 1
0 	 0
1 	 1
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
1 	 1
0 	 0
1 	 1
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
1 	 1
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
1 	 1
1 	 1
0 	 0
1 	 1
1 	 1
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
0 	 0
1 	 1
0 	 0
0 	 0
1 	 1
0 	 0
0 	 0
0 	 0
0 	 0
1 	 1
0 	 0
0 	 0
1 	 1
0 	 0
0 	 0
0 	 0
0 	 

100 % Correct rate?
Maybe in the features there are some feature have very strong relation with match.